# 실기 문제: 윤년 구하기

두 개의 년월일을 입력 받아, 입력 받은 기간 중, 윤년의 2월 29일의 요일, 실행시간, 윤년의 총 횟수를 출력하는 코드를 아래의 실행 예를 참고하여 객체지향프로그래밍으로 작성하시오.

## 실행 조건

- OOP의 특성을 잘 반영할 것.
- 입력 받은 날짜의 오류도 확인할 것.

## 참고 사항

윤년을 구하는 공식은 아래를 참고할 것.

- 4로 나누어떨어지는 해는 윤년이다.
- 100으로 나누어떨어지는 해는 윤년이 아니다.
- 400으로 나누어떨어지는 해는 윤년이다.

또한, 1년 1월 1일은 **월요일**이다.

## 실행 예 1

```
>>기간 시작 년월일을 입력하시오.
__년도를 입력하시오 : 2001
__월을 입력하시오 : 1
__일을 입력하시오 : 1
>>기간 종료 년월일을 입력하시오.
__년도를 입력하시오 : 2021
__월을 입력하시오 : 11
__일을 입력하시오 : 1
기간중 2004년의 2월 29일은 일요일 입니다.
기간중 2008년의 2월 29일은 금요일 입니다.
기간중 2012년의 2월 29일은 수요일 입니다.
기간중 2016년의 2월 29일은 월요일 입니다.
기간중 2020년의 2월 29일은 토요일 입니다.
실행된 총 시간: 0.002998828887939453 초.
기간 중 총 윤년의 횟수는 5 번 입니다.
```

## 실행 예 2

```
>>기간 시작 년월일을 입력하시오.
__년도를 입력하시오 : 1991
__월을 입력하시오 : 3
__일을 입력하시오 : 1
>>기간 종료 년월일을 입력하시오.
__년도를 입력하시오 : 2001
__월을 입력하시오 : 2
__일을 입력하시오 : 29
^^잘못된 입력입니다. 다시 입력하세요.
__년도를 입력하시오 : 2001
__월을 입력하시오 : 3
__일을 입력하시오 : 1
기간중 1992년의 2월 29일은 토요일 입니다.
기간중 1996년의 2월 29일은 목요일 입니다.
기간중 2000년의 2월 29일은 화요일 입니다.
실행된 총 시간: 0.0010137557983398438 초.
기간 중 총 윤년의 횟수는 3 번 입니다.
```

In [1]:
# Benchmark decorator


import time


def benchmark(f):
    """Record the execution time of the function.
    
    :param f: The function it decorates.
    :return: The 2-tuple of the original return value with the execution time in seconds.
    """
    def wrapper(*args, **kwargs):
        start_timestamp = time.perf_counter()
        result = f(*args, **kwargs)
        end_timestamp = time.perf_counter()
        return (result, end_timestamp - start_timestamp)
    
    return wrapper

In [2]:
# Classes representing year or a leap year


class _Year:
    """A year."""
    
    MONTHLY_DAYS = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    
    WEEKDAY_NAMES = ["일요일", "월요일", "화요일", "수요일", "목요일", "금요일", "토요일"]
    
    @property
    def initial_weekday_index(self):
        if not hasattr(self, "_initial_weekday_index"):
            self._initial_weekday_index = (
                    (self.year * 365) 
                    + ((self.year - 1) // 4)
                    - ((self.year - 1) // 100)
                    + ((self.year - 1) // 400)
                ) % 7
            
        return self._initial_weekday_index
    
    def __init__(self, year, start_date = None, end_date = None):
        self.year = year
        self.start_date = start_date
        self.end_date = end_date
        
    def is_valid_date(self, date):
        # Sanity check (month)
        if date[0] < 1 or date[0] > 12:
            return False
        
        # Sanity check (day)
        if date[1] < 1 or date[1] > self.MONTHLY_DAYS[date[0] - 1]:
            return False
        
        # If start date is specified...
        if self.start_date:
            if date[0] < self.start_date[0]:
                return False
            elif date[0] == self.start_date[0] and date[1] < self.start_date[1]:
                return False
            
        # If end date is specified...
        if self.end_date:
            if date[0] > self.end_date[0]:
                return False
            elif date[0] == self.end_date[0] and date[1] > self.end_date[1]:
                return False
        
        # Passed everything, it must be valid
        return True
    
    def get_weekday(self, date):
        # Make sure the date makes sense
        if not self.is_valid_date(date):
            return None
        
        # Extract month and day
        month, day = date
        
        # From January 1st...
        weekday = self.initial_weekday_index
        
        # ...add each passing months...
        for i in range(month - 1):
            weekday += self.MONTHLY_DAYS[i]
            
        # ...and add the passing days
        weekday += day - 1
        
        # Look it up
        return self.WEEKDAY_NAMES[weekday % 7]


class _LeapYear(_Year):
    """A leap year."""
    
    MONTHLY_DAYS = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    
    def __init__(self, year, start_date = None, end_date = None):
        super().__init__(year, start_date, end_date)

In [3]:
# Helper functions


def is_leap_year(year):
    """Check if the given year is a leap year.
    
    :param year: The year to check.
    :return: True if the given year is a leap year.
    """
    return year % 4 == 0 and year % 100 != 0 \
        or year % 400 == 0


def make_year(year, start_date = None, end_date = None):
    """Make a year object."""
    
    if is_leap_year(year):
        return _LeapYear(year, start_date, end_date)
    
    return _Year(year, start_date, end_date)


def input_date():
    while True:
        year = int(input("__년도를 입력하시오 : "))
        year_obj = make_year(year)
        
        month = int(input("__월을 입력하시오 : "))
        day = int(input("__일을 입력하시오 : "))
        
        # Date looks good, return
        if year_obj.is_valid_date((month, day)):
            return (year, month, day)
        
        # Wrong date, get input again
        print("^^잘못된 입력입니다. 다시 입력하세요.")
        
        
@benchmark
def list_leap_years(years):
    leap_count = 0
    for year in years:
        # If February 29th is not valid, it means:
        # 1. it's not a leap year or
        # 2. the range starts after/ends before Feb 29th
        if not year.is_valid_date((2, 29)):
            continue
        
        print(f"기간중 {year.year}년의 2월 29일은 {year.get_weekday((2, 29))} 입니다.")
        leap_count += 1
    
    return leap_count

In [4]:
# Main interactive cell


start_date = input_date()
start_year = make_year(start_date[0], start_date[1:])
end_date = input_date()
end_year = make_year(end_date[0], end_date[1:])

years = [start_year]
for diff in range(1, end_date[0] - start_date[0]):
    years.append(make_year(start_date[0] + diff))
years.append(end_year)

leap_count, exec_time = list_leap_years(years)


print(f"실행된 총 시간: {exec_time} 초.")
print(f"기간 중 총 윤년의 횟수는 {leap_count} 번 입니다.")


__년도를 입력하시오 : 2001
__월을 입력하시오 : 1
__일을 입력하시오 : 1
__년도를 입력하시오 : 2021
__월을 입력하시오 : 2
__일을 입력하시오 : 29
^^잘못된 입력입니다. 다시 입력하세요.
__년도를 입력하시오 : 2021
__월을 입력하시오 : 3
__일을 입력하시오 : 1
기간중 2004년의 2월 29일은 일요일 입니다.
기간중 2008년의 2월 29일은 금요일 입니다.
기간중 2012년의 2월 29일은 수요일 입니다.
기간중 2016년의 2월 29일은 월요일 입니다.
기간중 2020년의 2월 29일은 토요일 입니다.
실행된 총 시간: 7.142400045268005e-05 초.
기간 중 총 윤년의 횟수는 5 번 입니다.
